In [1]:
import sys 
sys.path.append('..')
from utils.paths import make_dirs, POTATOES_LIST, CLUSTERS_DIR_JSON, QUARTERS_DICT, GREEDY_MODULARITY_DIR, HASHTAG_TOP_DIR_JSON, HASHTAG_TOP_DIR, HASHTAG_FREQUENCY_DIR_ALL, POTATOES_CLEANED_DIR, FORBIDDEN_HASHTAGS_LIST, POTATOES_HASHTAGS_CLEAN, QUARTER_RESULTS, POTATOES_DIR, QUARTERS_ALL_POSTS, HASHTAGS_PER_USER_DIR, HASHTAG_FREQUENCY_DIR, CONNECTED_COMPONENTS_DIR, CONNECTED_COMPONENTS_PUBLIC_SPACES_DIR, HASHTAG_FREQUENCY_PUBLIC_SPACES_DIR, HASHTAGS_PER_USER_PUBLIC_SPACES_DIR
from utils.utils import load_dataframes, load_dataframe, write_dict_to_csv, write_list_to_csv, write_df_to_csv
import pandas as pd
import os
from glob import glob
import json
import numpy as np

make_dirs()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sophiaknapp/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sophiaknapp/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Top hashtags for wordcloud

In [5]:
# dfs = load_dataframes(HASHTAG_TOP_DIR)

for quarter in POTATOES_LIST():
    
    # df = dfs[quarter]
    df = load_dataframe(HASHTAG_TOP_DIR, quarter)
    df = df[['hashtag', 'count', 'opacitysigmoidlog']]
    # highest_rank = max(df['rank'])
    # df['rank'] = df['rank'] / highest_rank


    df.columns= ['text', 'count', 'opacity']
    df['opacity'] = np.round(df['opacity'], decimals=2)

    path = os.path.join(HASHTAG_TOP_DIR_JSON, quarter + '.json')
    df.to_json(path, orient='records')

    


>> now loading: allach


KeyError: "['count'] not in index"

### Clusters for packed circles

In [9]:
## OLD
min_count = 17
res = '1.8'

path = os.path.join(GREEDY_MODULARITY_DIR, 'min_count_' + str(min_count) + '_res_' + res)

for quarter in POTATOES_LIST():
    path_quarter = os.path.join(path, quarter)
    files_quarter = sorted(glob(path_quarter + '/*_.csv'))    
    data = []

    df_uniqueness = load_dataframe(HASHTAG_TOP_DIR, quarter)
    df_uniqueness['hashtag'] = df_uniqueness['hashtag'].astype(str)
    
    for file in files_quarter:
        df = pd.read_csv(file)
        print(df[:5])

        df['hashtag'] = df['hashtag'].astype(str)
        df = df.merge(df_uniqueness[['hashtag', 'opacitysigmoidlog']], how='left', on='hashtag')

        df['opacity'] = np.round(df['opacitysigmoidlog'], decimals=2)

        df = df[['hashtag', 'count', 'opacity']]

        df = df.fillna(0)

        name = df.iloc[0].hashtag
        df.rename(columns={'hashtag': 'name'}, inplace=True)
        records = df.to_dict('records')
        data.append({'name':name, 'children': records})

    final_data = {"name": quarter, "children": data}

    out_path = os.path.join(CLUSTERS_DIR_JSON, quarter + '.json')
    with open(out_path, "w") as outfile:
        json.dump(final_data, outfile)

>> now loading: allach
     hashtag  count
0        art     34
1  porcelain     27
2  porzellan     21
3        ww2     20
4      kunst     20
   hashtag  count
0  bavaria     74
1   nature     66
2      089     48
3    sonne     42
4     würm     36
     hashtag  count
0       bier     27
1       food     19
2   foodporn     19
3     lecker     19
4  tradition     18
     hashtag  count
0     dachau    115
1  karlsfeld     91
2      party     23
     hashtag  count
0    fitness     27
1  lifestyle     22
2      evers     22
3     family     21
4      sport     20
            hashtag  count
0         germering     34
1           olching     27
2         starnberg     24
3  fürstenfeldbruck     23
4        gröbenzell     22
            hashtag  count
0        immobilien     29
1  immobilienmakler     25
2      architecture     24
3           wohnung     19
4              haus     18
       hashtag  count
0         laim     48
1    schwabing     44
2  nymphenburg     39
3     sendling   